In [1]:
import csv
import pickle
import datetime

In [2]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png','retina'}

In [3]:
data = pd.read_csv('data/data.csv')
data=data.iloc[:,1:]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6,7,8,10,12,13,14,15,18,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
new_order =  ['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'avgspeed', 'resultspeed', 'resultdir', 'TS', 'GR', 'RA', 'DZ', 'SN',
       'SG', 'GS', 'PL', 'FG+', 'FG', 'BR', 'UP', 'HZ', 'FU', 'DU', 'SQ', 'FZ',
       'MI', 'PR', 'BC', 'BL', 'VC' , 'units' ]
data = data[new_order]

In [5]:
from sklearn.model_selection import train_test_split
X, y = data.iloc[:,:-1], data.units
X_train, X_testm, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2018)

In [6]:
def fulldf(df):
    
    df.snowfall = df.snowfall.replace(["T","  T"],0.05)
    df.preciptotal = df.preciptotal.replace(["T","  T"],0.005)
    df.depart = pd.to_numeric(df.depart, errors='coerce')
    
    df = type_change_numeric(df,[ 'store_nbr', 'item_nbr', 'units', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'avgspeed', 'resultspeed', 'resultdir' ])
    df['date'] = pd.to_datetime(df['date'])
    df["week_day_name"] = df['date'].dt.weekday_name
    df["day_of_year"] = df['date'].dt.dayofyear
    df["year"] = df['date'].dt.year
    df["month"] = df["date"].dt.month
    
    
    for idx in range(5, 8):
        df.iloc[:,idx].fillna(df.groupby(["day_of_year","store_nbr"])[df.columns[idx]].\
                              transform('mean'), inplace=True)

    for idx in range(16, 23):
        df.iloc[:,idx].fillna(df.groupby(["day_of_year","store_nbr"])[df.columns[idx]].\
                              transform('mean'), inplace=True)
    add_depart1(df)
    
    return df

In [7]:
def itemdf(df, nbr):
    item_nbr = df[df.item_nbr == nbr]
    for idx in range(5, 8):
        item_nbr.iloc[:,idx].fillna(item_nbr.groupby(["day_of_year","store_nbr"])[item_nbr.columns[idx]].\
                          transform('mean'), inplace=True)
    
    for idx in range(16, 23):
        item_nbr.iloc[:,idx].fillna(item_nbr.groupby(["day_of_year","store_nbr"])[item_nbr.columns[idx]].\
                          transform('mean'), inplace=True)
    add_depart1(item_nbr)
    
    return item_nbr

In [8]:
def type_change_numeric(df, ls = []): 
    #ls에 있는 column name은 numeric형으로 바꾸지 않는다.
    cols = df.columns
    for i in cols:
        if i in ls:
            #df = df.replace(["M",None], '')
            df.snowfall = df.snowfall.replace(["T","  T"],0.05)
            df.preciptotal = df.preciptotal.replace(["T","  T"],0.005)
            df[i] = pd.to_numeric(df[i], errors='coerce')
        
    return df


In [9]:
def add_depart1(x):
    x.depart.fillna(x.tavg - x.groupby(["day_of_year","store_nbr"])["tavg"].transform('mean'),inplace = True)
    x.depart = x.depart.round(2)
    return x.sort_values(["store_nbr","date","item_nbr"])

In [10]:
train = pd.concat([X_train, y_train] ,axis = 1)

In [11]:
train = train.sort_values(by=['date', 'store_nbr', 'item_nbr']).reset_index(drop= True)

In [12]:
df = fulldf(train)

In [13]:
pivot_df = df.groupby(["item_nbr", "month"]).mean().reset_index().pivot(index =  "month", columns = "item_nbr", values = "units")
pivot_df.style.apply(lambda x: ["background: #F78181" if abs(v) == 0 else "" for v in x], axis = 1)

item_nbr,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111
month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.31746,56.1442,46.6613,1.42424,30.5965,61.27,1.04918,0.95,0.530303,0,0.0508475,2.4939,33.218,1.08065,0.819444,0.761194,1.1194,0.923664,0,18.5339,1.36735,82.0797,3.15385,10.4677,4.13433,0.90625,1.28571,1.21739,0.472727,0.378788,0.15942,1.18033,48.0746,23.7818,1.81356,0.560606,0.42623,26.2074,2.29032,54.5758,127.562,69.6473,0.925373,1.19718,100.758,0.664671,0.916667,1.32727,1.33099,0.558824,0.490909,1.44444,2.02817,0.357143,1.10909,0.941176,0.9375,1.38071,0.923077,0,0.40678,0.338983,18.5192,7.88525,41.3556,0.84375,0.119403,0.671429,0.482759,0.617647,0.671233,0.623188,0.821429,0,0,0.693548,0.421875,5.46269,0.892308,7.84058,0.0463918,0.0673575,0.0793991,0.816667,0,0.0757576,1.26087,1.02857,0.246154,0.481343,0.707692,14.1311,1.55072,3.00752,1,0.730159,0.791667,0.382353,0.0131579,0.366667,0.0401606,0.031339,0.0461538,0.738462,1.50794,2.59843,0,0
2,0.45,0.797619,0.903226,0.181818,51.48,48.4906,1.32812,37.6308,59.8793,0.916667,0.741935,0.65,0,0.0327869,2.44211,34.8093,0.481481,0.660377,0.474576,0.793651,1.04065,0,16.725,3.97297,77.6306,3.67742,10.4833,5.13115,0.55,1.14516,0.576923,0.298246,0.310345,0.741935,1.1,38.885,22.8319,1.4697,0.633333,0.345455,24.9076,6.39062,56.0508,118.813,66.9218,0.539683,2.46667,80.459,0.696774,1.85328,1.69658,1.14679,0.689655,0.303571,1.72581,1.38182,0.460317,1.54098,0.923077,0.681818,1.2033,0.711864,1.81081,0.396552,0.606557,18.0244,11.7959,40.027,1.44444,0,0.65625,0.392857,0.785714,0.0491803,0.529412,0.508475,0.285714,0.310345,0.75,0.563636,4.53333,0.483871,6.5,0.0268817,0.145161,0.17217,1.22581,0.034188,0.107143,1.14062,0.428571,0,0.615068,0.881356,13.8246,0,0.017094,0.70229,0.690909,0.516667,0.454545,0,0.134615,0.032,0.113772,0.00833333,0.614035,1.98387,2.15254,0,0
3,2.22581,3.53403,1.52459,0.637681,47.3356,37.7656,0.939394,29.7937,53.4243,1.13043,0.467742,0.863636,1.09677,0.708333,1.1215,28.8789,0.594203,0.403226,0.412698,0.787879,1.05303,0,17.7054,4.39583,79.8732,3.55385,9.37879,4.8254,0.892308,1.01724,0.661538,0.776119,0.589286,0,0.526316,41.6957,20.3834,0.25,0.0534351,0.0142857,23.2832,5.84375,51.2609,119.86,64.9313,0.464789,2.0678,87.1212,0,0.0830671,0.215686,0.926471,0.6875,0.323077,1.42857,1.29457,1.16667,1.23288,1.04286,0.65,1.13613,0.594203,9.475,0.75,0.493151,17.4318,10.5172,38.8193,1.25,0.887097,0.677966,0.544118,0.365079,2.11111,0.432836,0.610169,3.25806,1.69643,0.651515,0.323529,4.90476,0.646154,7.22388,0.0659898,0.122449,0.125561,1.06154,0.030303,0.0588235,1.04615,0.439394,0.836066,0.764552,0.765625,12.8769,0,0.030303,0.736842,0.652778,0.826087,0.694444,0.102941,0.275362,0.0195312,0.032,0,0.823529,1.68182,1.86179,0.173913,0
4,1.44262,2.34973,1.48529,0.587302,42.8138,42.5152,0.683333,26.2424,50.5593,0.677966,0.525424,0.478873,1.32812,0.377049,0.0130293,24.6036,0.484848,0.28125,0.387097,0.6,0.515625,0,14.2701,5.5814,78.0265,3.77966,9.6,5.44286,0.813559,0.833333,0.465517,0.58209,0.246154,0,0.153846,36.7308,17.2279,0,0,0,19.763,6.50909,45.6308,104.182,59.4008,0.397059,1.35821,67.1231,0,0.00355872,0.0238095,0.578512,0.402985,0.295775,1.39706,1.05217,0.909091,1.22807,0.928571,0.703125,0.839779,0.779412,7.18605,0.322581,0.380952,17.186,9.68966,34.1532,1.125,0.947368,0.462687,0.454545,0.432836,1.14925,0.421875,0.369863,1.62903,1.67797,1,0.338462,6.22642,0.287879,7.91803,0.0604396,0.0618557,0.211137,1.29688,0.188976,0.0307692,1.21053,0.590164,1.66667,0.762222,0.872727,9.26415,0,0,0.396947,0.34375,0.545455,0.625,0.030303,0.142857,0.0447154,0.0271003,0.104,0.677966,2.84746,2.37903,1.85714,0
5,1.19231,1.64362,1.04545,0.6,38.4768,31.8286,0

In [14]:
df['weekend'] = False
df.weekend[df['week_day_name'] == 'Sunday'] = True
df.weekend[df['week_day_name'] == 'Saturday'] = True
df.weekend[df['week_day_name'] == 'Monday'] = True

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [26]:
model_GLSAR2 = sm.GLSAR.from_formula("I(log_units**3) ~  C(store_nbr) + C(item_nbr) + scale(depart) + scale(dewpoint) + scale(preciptotal) + C(SN) + C(FG2) + C(weekend) + C(month) + C(year) + C(is_blackfriday)" , data = df, rho=2)
results_GLSAR2 = model_GLSAR2.fit()
print(results_GLSAR2.summary())

PatsyError: Error evaluating factor: NameError: name 'log_units' is not defined
    I(log_units**3) ~  C(store_nbr) + C(item_nbr) + scale(depart) + scale(dewpoint) + scale(preciptotal) + C(SN) + C(FG2) + C(weekend) + C(month) + C(year) + C(is_blackfriday)
    ^^^^^^^^^^^^^^^

In [18]:
data.head()

,date,store_nbr,item_nbr,station_nbr,tmax,tmin,tavg,depart,dewpoint,wetbulb,...,FU,DU,SQ,FZ,MI,PR,BC,BL,VC,units
0,2012-01-01,1,9,1,52,31,42,M,36,40,...,False,False,False,True,False,False,False,False,False,29.0
1,2012-01-01,1,28,1,52,31,42,M,36,40,...,False,False,False,True,False,False,False,False,False,2.0
2,2012-01-01,1,40,1,52,31,42,M,36,40,...,False,False,False,True,False,False,False,False,False,0.0
3,2012-01-01,1,47,1,52,31,42,M,36,40,...,False,False,False,True,False,False,False,False,False,0.0
4,2012-01-01,1,51,1,52,31,42,M,36,40,...,False,False,False,True,False,False,False,False,False,1.0


In [20]:
data["date"] = pd.to_datetime(data["date"])
data["year"] = data["date"].dt.year
data["month"] = data["date"].dt.month
data["quarter"] = data["date"].dt.quarter
data["week_day_name"] = data["date"].dt.weekday_name
data["day_of_year"] = data["date"].dt.dayofyear

In [21]:
data['weekend'] = False
data.weekend[data['week_day_name'] == 'Sunday'] = True
data.weekend[data['week_day_name'] == 'Saturday'] = True
data.weekend[data['week_day_name'] == 'Friday'] = True

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [24]:
blackfriday = ["2012-11-21","2012-11-22","2012-11-23", "2012-11-24","2012-11-25",
              "2013-11-27","2013-11-28", "2013-11-29","2013-11-30","2013-11-31",
              "2014-11-26", "2014-11-27", "2014-11-28","2014-11-29","2014-11-30"]

In [25]:
data["is_blackfriday"] = data.date.apply(lambda x : str(x)[:10] in blackfriday).astype(int)

In [34]:
data["log_units"] = data.units.apply(lambda x : np.log(x + 1))
# 특정조건열 추가 (0, 0.2 , 0.5 , 1 , 3 5단계로 강우량 단계 설정)
# data['hardrain'] = [(((4 if i > 3 else 3) if i > 1 else 2) if i > 0.5 else 1) if i > 0.1 else 0 for i in data['preciptotal']]

# # 득정조건열 추가 ()
# data['hardsnow'] = [(((4 if i > 3 else 3) if i > 2 else 2) if i > 1 else 1) if i > 0.2 else 0 for i in data['snowfall']]

TypeError: '>' not supported between instances of 'str' and 'float'

In [37]:
data = data[data.tmax.isnull() == False]
data = data[data.tmin.isnull() == False]
data = data[data.tavg.isnull() == False]
data = data[data.dewpoint.isnull() == False]
data = data[data.wetbulb.isnull() == False]
data = data[data.preciptotal.isnull() == False]
data = data[data.depart.isnull() == False]


In [38]:
data.dtypes

date              datetime64[ns]
store_nbr                  int64
item_nbr                   int64
station_nbr                int64
tmax                      object
tmin                      object
tavg                      object
depart                    object
dewpoint                  object
wetbulb                   object
heat                      object
cool                      object
sunrise                   object
sunset                    object
snowfall                  object
preciptotal               object
stnpressure               object
sealevel                  object
avgspeed                  object
resultspeed               object
resultdir                 object
TS                          bool
GR                          bool
RA                          bool
DZ                          bool
SN                          bool
SG                          bool
GS                          bool
PL                          bool
FG+                         bool
FG        

In [35]:
model_GLSAR2 = sm.GLSAR.from_formula("I(log_units**3) ~  C(store_nbr):C(item_nbr)  + scale(dewpoint) + scale(preciptotal) + C(SN) + C(FG2) + C(weekend) + C(month) + C(year) + C(is_blackfriday)" , data = data, rho=2)
results_GLSAR2 = model_GLSAR2.fit()
print(results_GLSAR2.summary())

PatsyError: Error evaluating factor: ValueError: cannot widen a non-numeric type dtype('O')
    I(log_units**3) ~  C(store_nbr) + C(item_nbr) + scale(depart) + scale(dewpoint) + scale(preciptotal) + C(SN) + C(FG2) + C(weekend) + C(month) + C(year) + C(is_blackfriday)
                                                                                      ^^^^^^^^^^^^^^^^^^